This notebook generates bounding boxes for each of the classes. The class information is provided in polygon form in `train_wkt_v4.csv` and is scaled according to the image sizes `60##_#_#_{}.tiff` and grid sizes in `grid_sizes.csv`.  It produces both a `.npz` containing a dict of dict of bboxes for each image. It will then also write these to an appropriate form for the DL algorithm of our choice, currently Faster R-CNN. The XML format is copied from there. Also produces `.txt` files indicating which images belong to which dataset (train, val, test).

If you intend to reproduce this, you might want to attach 'my' external volume to your AWS session.

First import the necessary modules and set the global variables N_CLASSES, BAND and MASK_SIZES.

In [ ]:
import csv
import sys
import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import numpy as np
import tifffile as tiff
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 10)
import itertools
csv.field_size_limit(sys.maxsize)
import h5py
import os

import helper # this module contains some function that are used by a few of the notebooks on this matter

N_VAL = 5
N_CLASSES = 10
BAND = 'RGB' # which band to use
MASK_SIZES = {'A': 128, 'M': 800, 'P': 3*1024, 'RGB': 1024} # size of the Mask ARRAY to use
DEPTHS = {'A': 8, 'M': 8, 'P': 1, 'RGB': 3}
data_dir = '/data/dstl'
if BAND == 'RGB':
    dir_name = os.path.join(data_dir, 'three_band')
else:
    dir_name =  os.path.join(data_dir, 'sixteen_band')
class_names = {0: 'buildings', 
               1: 'misc. manmade structures',
               2: 'road',
               3: 'track',
               4: 'trees',
               5: 'crops',
               6: 'waterway',
               7: 'standing water',
               8: 'vehicle large',
               9: 'vehicle small'}

Now define and run a function that fetches the size of each image and the scaler (x and y dir) required for scaling the polygons to the correct shape.

In [ ]:
def create_sizes_and_scalers():
    im_sizes = {}
    scalers = {}
    
    for im_id in helper.image_iterator():
        masks_per_im = []
        im = tiff.imread(os.path.join(dir_name, 'TIFFImages', '{}_{}.tiff'.format(im_id, BAND)))
        im_size = im.shape[-2::] # last two, regardless of whether there are three dims or not
        im_sizes[im_id] = im_size

    with open(os.path.join(data_dir, 'grid_sizes.csv')) as inf:
        reader = csv.reader(inf)
        next(reader)
        for im_id, x, y in reader:
            print im_id
            scaler = np.array(im_sizes[im_id])
            scaler = scaler * scaler.astype(float) / (scaler + 1)
            scalers[im_id] = (scaler[1]/float(x), scaler[0]/float(y))
    return im_sizes, scalers

im_sizes, scalers = create_sizes_and_scalers()

Fill the masks with binary pixels indicating whether the pixel is in a certain class or not. We only create a dict entry for images that contain any class data. 

In [ ]:
def create_bboxes(im_sizes, scalers):
    bboxes = {}

    with open(os.path.join(data_dir, 'train_wkt_v4.csv')) as inf:
        train = csv.reader(inf)
        next(train) # skip the header row
        for row_i, (im_id, poly_type, wktpoly) in enumerate(train):
            poly_type = int(poly_type) - 1
            int_coords = lambda x: np.array(x).round().astype(np.int32)
            polygons = shapely.wkt.loads(wktpoly)
            x_scaler, y_scaler = scalers[im_id]
            polygons = shapely.affinity.scale(polygons, xfact=x_scaler, yfact=y_scaler, origin=(0, 0, 0))

            if polygons:
                exteriors = [int_coords(poly.exterior.coords) for poly in polygons]

                # create masks for this image if they do not yet exist
                if not im_id in bboxes:
                    bboxes[im_id] = {}
                    for c in range(N_CLASSES):
                        bboxes[im_id][c] = []

                for exterior in exteriors:
                    x0 = exterior[:, 0].min()
                    x1 = exterior[:, 0].max()
                    y0 = exterior[:, 1].min()
                    y1 = exterior[:, 1].max()

                    # store bbox in both representations for what is necessary
                    bbox = {'x0': x0, 'x1': x1, 'y0': y0, 'y1': y1, 
                            'x': (x0+x1)/2, 'y': (y0+y1)/2, 'w': (x1-x0), 'h': (y1-y0)}
                    bboxes[im_id][poly_type].append(bbox)
                print im_id, poly_type, len(bboxes[im_id][poly_type])
         
                # some output to keep track of the progress
    print 'Completed'
    return bboxes

bboxes = create_bboxes(im_sizes, scalers)

Save the result to bboxes_{}.npz

In [ ]:
np.savez(os.path.join(data_dir, 'bboxes_{}.npz'.format(BAND)), bboxes=bboxes, im_sizes=im_sizes)

Load the data from the drive (only if not already in memory)

In [ ]:
bboxes = np.load(os.path.join(data_dir, 'bboxes_{}.npz'.format(BAND)))['bboxes'][()]

Loop over all images, but tell iterator to only produce ids that have masks.
Note that this could also be done via a loop over the masks, but we previously also included empty data, and might wish to so again in the future

In [ ]:
id_val = []
id_trn = []
id_tst = []
for i, im_id in enumerate(helper.image_iterator(bboxes.keys())):
    pass
N_tot = i
N_lbl = len(bboxes)
np.random.seed(21)
choice = np.sort(np.random.choice(range(N_lbl), N_VAL, replace=False))

i = 0
for im_id in helper.image_iterator():
    if im_id in bboxes:
        if i in choice:
            id_val.append(im_id)
        else:
            id_trn.append(im_id)
        i = i + 1
    else:
        id_tst.append(im_id)

In [ ]:
list_dir = os.path.join(dir_name, 'ImageSets', 'Main')
with open(os.path.join(list_dir,'train.txt'), 'w') as f_trn, \
     open(os.path.join(list_dir, 'val.txt'), 'w') as f_val, \
     open(os.path.join(list_dir, 'trainval.txt'), 'w') as f_trnval, \
     open(os.path.join(list_dir, 'test.txt'), 'w') as f_tst:
    for im_id in id_val:
        f_val.write('{:s}_{:s}\n'.format(im_id, BAND))
        f_trnval.write('{:s}_{:s}\n'.format(im_id, BAND))
    for im_id in id_trn:
        f_trn.write('{:s}_{:s}\n'.format(im_id, BAND))
        f_trnval.write('{:s}_{:s}\n'.format(im_id, BAND))
    for im_id in id_tst:
        f_tst.write('{:s}_{:s}\n'.format(im_id, BAND))
            

With all the work done, let us inspect some of the results

In [ ]:
def show(img, interpolation='none', ax=None, **kwargs):
    if ax is None:
        fig, ax = plt.subplots(1, 1)
    ax.imshow(img, interpolation='none', **kwargs)
    return ax

def draw_rectangle(bbox, ax=None, color=None):
    x, y, w, h = bbox['x'], bbox['y'], bbox['w'], bbox['h']
    x_v = np.array([-.5,  .5, .5, -.5, -.5])
    y_v = np.array([-.5, -.5, .5,  .5, -.5])
    if ax is None:
        fig, ax = plt.subplots(1, 1)
    ax.plot(x + w*x_v, y + h*y_v, '-', color=color)

In [ ]:
i_val = 4
im_id = id_val[i_val] # Let's look at the first image in the validation set
im = tiff.imread(os.path.join(dir_name, 'TIFFImages', '{}_{}.tiff'.format( im_id, BAND)))
# im = im.transpose([1, 2, 0]).astype(float)/(8*255)
im = (im.transpose([1, 2, 0])/8).astype(np.uint8)
show(im)


In [ ]:
def get_color(class_name):
    import colorsys
    h = (hash(class_name) % np.pi) / np.pi
    # v = (hash(class_name) % 10)/20. + .5
    N_v = 3
    v = .5/(N_v-1)*np.floor((hash(class_name) % (N_v*np.pi))/np.pi) + .5
    return colorsys.hsv_to_rgb(h, .8, v)


In [ ]:
c = 0# class
im_annotate = im.copy()
print len(bboxes[im_id][c])
ax = show(im_annotate)
for i, bbox in enumerate(bboxes[im_id][c]):
    draw_rectangle(bbox, ax, get_color(class_names[c]))

In [ ]:
if 'masks' not in globals():
    masks = np.load(os.path.join(data_dir, 'masks_{}.npz'.format(BAND)))['masks'][()]

In [ ]:
show(masks[im_id][c], cmap='gray')

The loop below shows four images per row, one for each class: the bare image, the image overlaid with bounding boxes, the masks from the `mask` dictionary and the masks from the `y_val` tensor.

In [ ]:
fig, axs = plt.subplots(10, 4, sharex=True, sharey=True)
fig.set_size_inches(10, 24)

im = tiff.imread(os.path.join(dir_name, 'TIFFImages', '{}_{}.tiff'.format(im_id, BAND)))
im = (im.transpose([1, 2, 0])/8).astype(np.uint8)

X_val, y_val, X_trn, y_trn = helper.get_more_patches(BAND, 1024, repetitions=1, classes=range(10))

for c, class_name in enumerate(classes):
    im_annotate = im.copy()
    show(im, ax=axs[c, 0])
    show(im_annotate, cmap='gray', ax=axs[c, 1])
    for i, bbox in enumerate(bboxes[im_id][c]):
        draw_rectangle(bbox, ax=axs[c, 1], color=get_color(class_name))
    show(masks[im_id][c], cmap='gray', ax=axs[c, 2])
    show(y_val[i_val, c], cmap='gray', ax=axs[c, 3])

In [ ]:
import time

fig, axs = plt.subplots(3, 4)
axs = axs.flatten()
counts = np.zeros(len(class_names), dtype=int)

t_next = time.time()
# for im_id, bboxes_per_im in bboxes.items():
for im_id in id_val:
    bboxes_per_im = bboxes[im_id]
    print im_id,
    for ax , (c, class_name) in zip(axs, class_names.items()):
        for bbox in bboxes_per_im[c]:
            draw_rectangle(bbox, ax, get_color(class_names[c]))
        counts[c]+= len(bboxes_per_im[c])
    t_prev, t_next = t_next, time.time()
    print counts.sum(), '\t', t_next - t_prev

In [ ]:
plt.savefig('all_bboxes.png')

In [ ]:
for count, class_name in zip(counts, class_names.values()):
    print "{:s}: {:d}, average {:d} per image".format(class_name, count, count/len(bboxes))

In [ ]:
edges = 2**np.arange(0, 24+1)

mids = (edges[:-1] + edges[1:])/2
N_hist = len(edges)-1
hist = np.zeros((len(class_names), N_hist), dtype=int)

t_next = time.time()
# for im_id, bboxes_per_im in bboxes.items():
for im_id in id_trn:
    bboxes_per_im = bboxes[im_id]
    print im_id,
    for (c, class_name) in class_names.items():
        for bbox in bboxes_per_im[c]:
            area = bbox['w'] * bbox['h']
            i = (edges[1:] < area).sum()
            if i < N_hist:
                hist[c, i]+=1
            else:
                print "Found bbox too large ({:d}) for class {:s}".format(area, class_name)
    t_prev, t_next = t_next, time.time()
    print hist.sum(), '\t', t_next - t_prev

In [ ]:
plt.loglog(mids, hist.T, '.-')
plt.legend(class_names.values())

In [ ]:
for c, class_name in class_names.items():
    min_area = edges[:-1][hist[c, :] > 0][0]
#     print "{:s} has smallest bbox size of ±{:d} = 2**{:d}".format(class_name, min_area, int(np.log2(min_area)))
    max_area = edges[1:][hist[c, :] > 0][-1]
#     print "{:s} has largest bbox size of ±{:d} = 2**{:d}".format(class_name, max_area, int(np.log2(max_area)))
    print "{:s} has range log2 of {:d}--{:d}".format(class_name, int(np.log2(min_area)), int(np.log2(max_area)))

In [ ]:
edges

In [ ]:
plt.loglog(mids, hist.sum(axis=1) - hist.T.cumsum(axis=0), '.-')
plt.legend(class_names.values())
plt.figure()
plt.loglog(mids, hist.T.cumsum(axis=0), '.-')
plt.legend(class_names.values())